# ML Pipeline Test - Multiple Models Comparison
This notebook demonstrates the usage of our ML pipeline module with multiple models.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from src.main import run_model_pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Font
from src.Exel_Modules.excel_utils import save_predictions_and_metrics_to_excel



## Define Models
We'll test multiple models with different configurations.

# Define models to test

In [3]:

models = {
    'Random Forest': {
        'library': 'sklearn.ensemble',
        'function': 'RandomForestClassifier',
        'params': {'n_estimators': 100, 'max_depth': 10}
    },
    'Gradient Boosting': {
        'library': 'sklearn.ensemble',
        'function': 'GradientBoostingClassifier',
        'params': {'n_estimators': 100, 'learning_rate': 0.1}
    },
    'SVM': {
        'library': 'sklearn.svm',
        'function': 'SVC',
        'params': {'kernel': 'rbf', 'C': 1.0}
    },
    'Logistic Regression': {
        'library': 'sklearn.linear_model',
        'function': 'LogisticRegression',
        'params': {'max_iter': 1000}
    }
}

In [4]:


# Data parameters
DATA_PATH = "data/FraudDetectionDataset.xlsx"
TARGET_COLUMN = "Fraudulent"

# Store results for all models
all_results = {}

# Run pipeline for each model
for model_name, model_config in models.items():
    print(f"\nTraining {model_name}...")
    results = run_model_pipeline(
        data_path=DATA_PATH,
        target_column=TARGET_COLUMN,
        model_library=model_config['library'],
        model_function=model_config['function'],
        model_params=model_config['params']
    )
    all_results[model_name] = results
# print(all_results["Random Forest"])


Training Random Forest...
[Train] Accuracy: 0.9683 | Precision: 0.9702 | Recall: 0.9659 | f1_score: 0.9680
[Test ] Accuracy: 0.4712 | Precision: 0.4560 | Recall: 0.4500 | f1_score: 0.4530
[All  ] Accuracy: 0.8688 | Precision: 0.8698 | Recall: 0.8644 | f1_score: 0.8671

Training Gradient Boosting...
[Train] Accuracy: 0.7039 | Precision: 0.7113 | Recall: 0.6811 | f1_score: 0.6959
[Test ] Accuracy: 0.5083 | Precision: 0.4945 | Recall: 0.4737 | f1_score: 0.4839
[All  ] Accuracy: 0.6648 | Precision: 0.6686 | Recall: 0.6403 | f1_score: 0.6542

Training SVM...
[Train] Accuracy: 0.6302 | Precision: 0.6376 | Recall: 0.5941 | f1_score: 0.6151
[Test ] Accuracy: 0.4930 | Precision: 0.4781 | Recall: 0.4605 | f1_score: 0.4692
[All  ] Accuracy: 0.6028 | Precision: 0.6054 | Recall: 0.5678 | f1_score: 0.5860

Training Logistic Regression...
[Train] Accuracy: 0.5309 | Precision: 0.5299 | Recall: 0.5019 | f1_score: 0.5156
[Test ] Accuracy: 0.5122 | Precision: 0.4986 | Recall: 0.4684 | f1_score: 0.4830
[

# ==== Save Predictions to Excel ====

In [ ]:


# Call the function
save_predictions_and_metrics_to_excel(
    results_dict=all_results,
    model_name="Random Forest",
    excel_path="data/FraudDetectionDataset.xlsx"
)


In [82]:

# Your Excel path
DATA_PATH = "data/FraudDetectionDataset.xlsx"

# Get everything from all_results
result_all = all_results["Random Forest"]["all"]
n_train = len(all_results["Random Forest"]["train"]["true_values"])

# Extract true and predicted values
all_true_values = result_all["true_values"]
all_predictions = result_all["predictions"]

# Separate into train and test labels
data_labels = ["train"] * n_train + ["test"] * (len(all_true_values) - n_train)

# Extract metrics from the same dict (excluding lists)
metrics = {k: v for k, v in result_all.items() if k not in ["predictions", "true_values"]}


# === 1. Create dataframe ===


In [83]:
df_combined = pd.DataFrame({
    "ID": range(1, len(all_true_values) + 1),
    "True": all_true_values,
    "Predicted": all_predictions,
    "Set": data_labels
})

# Save prediction table (without Set column)
with pd.ExcelWriter(DATA_PATH, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_combined.drop(columns=["Set"]).to_excel(writer, sheet_name="RF", index=False)


# === 2. Styling and metric table ===


In [84]:


wb = load_workbook(DATA_PATH)
ws = wb["RF"]

# Styling
train_fill = PatternFill(start_color="DFF0D8", end_color="DFF0D8", fill_type="solid")
test_fill = PatternFill(start_color="F2DEDE", end_color="F2DEDE", fill_type="solid")
center_align = Alignment(horizontal="center", vertical="center")
bold_font = Font(bold=True)

# Get column index of "Predicted"
header = [cell.value for cell in ws[1]]
predicted_col_index = header.index("Predicted") + 1

# Apply styles and color only to Predicted column
for row_idx, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), start=0):
    for col_idx, cell in enumerate(row, start=1):
        cell.alignment = center_align
        if col_idx == predicted_col_index:
            fill = train_fill if data_labels[row_idx] == "train" else test_fill
            cell.fill = fill

# Style header
for cell in ws[1]:
    cell.alignment = center_align
    cell.font = bold_font


# === 3. Add metrics table to the right of predictions ===


In [85]:
from openpyxl.styles import PatternFill, Alignment, Font
from openpyxl.utils import get_column_letter

def write_metrics_table(ws, metrics_dict, start_row, start_col, fill_color, label):
    center_align = Alignment(horizontal="center", vertical="center")
    bold_font = Font(bold=True)
    fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")

    # Write label merged across the number of metrics columns
    n_cols = len(metrics_dict)
    end_col_letter = get_column_letter(start_col + n_cols - 1)
    ws.merge_cells(start_row=start_row, start_column=start_col, end_row=start_row, end_column=start_col + n_cols - 1)
    label_cell = ws.cell(row=start_row, column=start_col, value=label)
    label_cell.font = Font(bold=True, color="FFFFFF")
    label_cell.alignment = center_align
    label_cell.fill = fill

    # Write headers below label
    header_row = start_row + 1
    for i, header in enumerate(metrics_dict.keys()):
        cell = ws.cell(row=header_row, column=start_col + i, value=header.capitalize())
        cell.font = bold_font
        cell.alignment = center_align
        cell.fill = fill

    # Write values below headers
    value_row = header_row + 1
    for i, value in enumerate(metrics_dict.values()):
        cell = ws.cell(row=value_row, column=start_col + i, value=round(value, 4))
        cell.alignment = center_align

# Usage example:

metrics_all = all_results["Random Forest"]["all"]
metrics_train = all_results["Random Forest"]["train"]
metrics_test = all_results["Random Forest"]["test"]

start_col = ws.max_column + 1
start_row = 1

write_metrics_table(ws, 
                    {k: v for k, v in metrics_all.items() if k not in ["predictions", "true_values"]}, 
                    start_row, start_col, "ADD8E6", "All Metrics")

write_metrics_table(ws, 
                    {k: v for k, v in metrics_train.items() if k not in ["predictions", "true_values"]}, 
                    start_row + 5, start_col, "90EE90", "Train Metrics")

write_metrics_table(ws, 
                    {k: v for k, v in metrics_test.items() if k not in ["predictions", "true_values"]}, 
                    start_row + 10, start_col, "FFC0CB", "Test Metrics")


wb.save(DATA_PATH)
